# IMDB Investigation

### Import Library

In [5]:
# import
import numpy as np
import pandas as pd


### Import Data

In [6]:
basics_url='https://datasets.imdbws.com/title.basics.tsv.gz'
akas_url='https://datasets.imdbws.com/title.akas.tsv.gz'
ratings_url='https://datasets.imdbws.com/title.ratings.tsv.gz'

basics = pd.read_csv("Data/title_basics.csv.gz", low_memory=False)
akas = pd.read_csv("Data/title_akas.csv.gz",  low_memory=False)
ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory=False)

### Saving to file

In [7]:
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)
akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)
ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)

### Review Dataframes

In [8]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [9]:
#creating backup of 'basics' dataframe
df_b_save = basics.copy()

In [10]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [11]:
df_a_save = akas.copy()

In [12]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1977
1,tt0000002,5.8,264
2,tt0000003,6.5,1829
3,tt0000004,5.6,179
4,tt0000005,6.2,2620


In [13]:
df_r_save = ratings.copy()

### Preprocessing Cleanup

#### title.basics

##### Find NAN Values

In [14]:
basics = basics.replace({'\\N':np.nan})

##### Evaluating NAN values in columns

In [15]:
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle           11
originalTitle          11
isAdult                 1
startYear         1335463
endYear           9792973
runtimeMinutes    6975078
genres             444872
dtype: int64

In [16]:
#Display the percentage of missing values by row. 
print(basics.isna().sum()/len(basics)*100)

tconst             0.000000
titleType          0.000000
primaryTitle       0.000111
originalTitle      0.000111
isAdult            0.000010
startYear         13.488905
endYear           98.914373
runtimeMinutes    70.452095
genres             4.493450
dtype: float64


##### Dropping column endYear since this primarily an investigation of movies

In [17]:
basics.drop(['endYear'], axis=1, inplace=True)

##### Eliminate movies that are null for runtimeMinutes

In [18]:
basics.dropna(subset=['runtimeMinutes'], inplace=True)

##### Eliminate movies that are null for genre

In [19]:
basics.dropna(subset=['genres'], inplace=True)

##### keep only titleType==movie

In [20]:
basics = basics[basics['titleType'] == 'movie']

##### Eliminating NAN values for 'startYear'

In [21]:
basics.dropna(subset=['startYear'], inplace=True)

##### NAN values resolved

In [22]:
print(basics.isna().sum()/len(basics)*100)

tconst            0.0
titleType         0.0
primaryTitle      0.0
originalTitle     0.0
isAdult           0.0
startYear         0.0
runtimeMinutes    0.0
genres            0.0
dtype: float64


##### Modifying Datatypes to conform to data dictionary

In [23]:
basics = basics.astype({"tconst":"string","titleType":"string","primaryTitle":"string","originalTitle":"string", "isAdult":"int", "startYear":"int","runtimeMinutes":"int", "genres":"string"})

In [24]:
basics.dtypes

tconst            string
titleType         string
primaryTitle      string
originalTitle     string
isAdult            int32
startYear          int32
runtimeMinutes     int32
genres            string
dtype: object

##### keep startYear 2000-2022

In [25]:
basics = basics[(basics['startYear'] >= 2000) & (basics['startYear'] <= 2022)]

##### Eliminate movies that include "Documentary" in genre

In [26]:
basics = basics[~basics['genres'].str.contains(r'documentary', case=False)]


#### title.akas

##### Find NAN Values

In [27]:
akas =akas.replace({'\\N':np.nan})

##### Filtering out all non US Films

In [28]:
akas = akas[akas['region'] == 'US']

##### Evaluating NAN values in columns

In [29]:
akas.isna().sum()

titleId                  0
ordering                 0
title                    0
region                   0
language           1437744
types               462005
attributes         1395001
isOriginalTitle       1345
dtype: int64

In [30]:
print(akas.isna().sum()/len(akas)*100)

titleId             0.000000
ordering            0.000000
title               0.000000
region              0.000000
language           99.726501
types              32.046138
attributes         96.761710
isOriginalTitle     0.093293
dtype: float64


##### Dropping Columns Language and Attributes due to lack of data, as well as isOriginalTitle since it doesnt directly relate to investigation

In [31]:
akas.drop(['language','attributes','isOriginalTitle'], axis=1, inplace=True)

##### Replacing NAN values with Unknown for types column

In [32]:
type_of = 'unknown'
akas['types'].fillna(value = type_of, inplace = True)

##### NAN Values Resolved

In [33]:
akas.isna().sum()

titleId     0
ordering    0
title       0
region      0
types       0
dtype: int64

##### Modifying Datatypes to conform to Data Dictionary

In [34]:
akas = akas.astype({"titleId":"string","title":"string","region":"string","types":"string"})

In [35]:
akas.dtypes

titleId     string
ordering     int64
title       string
region      string
types       string
dtype: object

#### title.ratings

##### Find NAN Values

In [36]:
ratings = ratings.replace({'\\N':np.nan})

In [37]:
ratings.isna().sum()

tconst           0
averageRating    0
numVotes         0
dtype: int64

* No NAN values found

##### Modifying Datatypes to conform to data dictionary

In [38]:
ratings = ratings.astype({"tconst":"string"})

In [39]:
ratings.dtypes

tconst            string
averageRating    float64
numVotes           int64
dtype: object

#### Filtering for Title Basics with akas dataframe for US only

In [40]:
keepers1 =basics['tconst'].isin(akas['titleId'])
keepers1

34803       True
42384       True
61115       True
67668       True
86800       True
           ...  
9900128     True
9900137     True
9900176    False
9900221     True
9900305    False
Name: tconst, Length: 147654, dtype: bool

In [41]:
basics = basics[keepers1]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,118,"Comedy,Fantasy,Romance"
42384,tt0043139,movie,Life of a Beijing Policeman,Wo zhe yi bei zi,0,2013,120,"Drama,History"
61115,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,70,Drama
67668,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,122,Drama
86800,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,100,"Comedy,Horror,Sci-Fi"
...,...,...,...,...,...,...,...,...
9899593,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,74,Drama
9899988,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019,97,"Comedy,Drama,Fantasy"
9900128,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,51,Drama
9900137,tt9916190,movie,Safeguard,Safeguard,0,2020,95,"Action,Adventure,Thriller"


#### Filtering for ratings with akas dataframe for US only

In [42]:
keepers2 =ratings['tconst'].isin(akas['titleId'])
keepers2

0           True
1           True
2          False
3          False
4           True
           ...  
1318849    False
1318850    False
1318851    False
1318852    False
1318853    False
Name: tconst, Length: 1318854, dtype: bool

In [43]:
ratings = ratings[keepers2]
ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,1977
1,tt0000002,5.8,264
4,tt0000005,6.2,2620
5,tt0000006,5.1,182
6,tt0000007,5.4,821
...,...,...,...
1318815,tt9916200,8.1,229
1318816,tt9916204,8.2,262
1318823,tt9916348,8.3,18
1318824,tt9916362,6.4,5367


### Saving modified dataframes to file for later use

In [44]:
basics.to_csv("Data/title_basics1.csv.gz",compression='gzip',index=False)
akas.to_csv("Data/title_akas1.csv.gz",compression='gzip',index=False)
ratings.to_csv("Data/title_ratings1.csv.gz",compression='gzip',index=False)